<img src="https://www.mbitschool.com/wp-content/uploads/2020/08/LOGO.png" alt="MarineGEO circle logo" style="width:100px;"/>

# MBIT School

## Máster Online en Data Science

---

## Apache Spark: Análisis de datos en Big Data

---

#### Pedro Gómez (pegomez.lopez@mbitschool.com)

---

---

##  Proyecto de consolidación - Dataset Amazon Reviews

En la siguiente URL podemos encontrar un dataset público de amazon que contiene una gran cantidad de reviews generadas a lo largo de los años.

El dataset se encuentra almacenado en un bucket público y podemos consumirlo desde nuestros nodos de EMR

https://s3.amazonaws.com/amazon-reviews-pds/readme.html

<div style="text-align: center">
<h1><font color="black" size=8>PROYECTO DE CONSOLIDACIÓN</font></h1>

<div style="text-align: left">
<h1><font color="#009D7F" size=5>Alejandro Santiago Bitria</font></h1>

## Ejercicios: Responde a las siguientes preguntas usando Spark

In [1]:
# Iniciamos la sesión de Spark
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1650391259875_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Cargamos las librerías

Antes de nada, procedo a cargar todas las librerias necesarias para este Proyecto.

In [26]:
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import split, size
from pyspark.sql.functions import col, count, avg, sum, round, mean, max, min
from pyspark.sql.functions import year, month

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Carga del dataset y primeros checks

Cargar desde amazon S3 el dataset en su version parquet y verificar el esquema, muestra también las primeras filas del mismo

In [3]:
df_amazon = (spark.read.option("inferSchema", True).load("s3://amazon-reviews-pds/parquet/"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_amazon.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+----------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|year|product_category|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+----------------+
|         US|   15444933|R1WWG70WK9VUCH|1848192576|     835940987|Standing Qigong f...|          5|            9|         10|   N|                Y|Informative AND i...|After attending a...| 2015-05-02|2015|           Books|
|         US|   20595117|R1EQ3POS0RIOD5|145162445X|     574044348|A Universe from N...|          4| 

### Pregunta 1: ¿Hay algo en el esquema que llame tu atención? ¿Realizarías alguna modificación?

Lo primero que comprobamos es si tiene valores nulos.

In [46]:
df_amazon.select([count(when(col(c).isNull(), c)).alias(c) for c in df_amazon.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+----------------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|year|product_category|
+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+----------------+
|          0|          0|        0|         0|             0|            0|          0|            0|          0|   0|                0|            320|       6798|          0|   0|               0|
+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+----------------+

Parece ser que hay valores nulos en `review_headline` y en `review_body`. Para comprobar si es necesario eliminarlas o si su volumen no es mucho en comparación al total vamos a sacar los respectivos porcentajes de valores nulos.

In [22]:
# Porcentaje de review_headline

percentage_null_review_headline = (df_amazon.filter("review_headline is null").count()/df_amazon.count()*100)
percentage_null_review_headline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.00019900922015936035

In [7]:
# Porcentaje de review_body

percentage_null_review_body = (df_amazon.filter("review_body is null").count()/df_amazon.count()*100)
percentage_null_review_body

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.004227702120760412

Ninguno de estos porcentajes **siquiera llega al 0,01%** por lo que podemos eliminarlos sin que se pierda apenas información de este dataset.

In [5]:
# Eliminamos cualquier valor nulo del dataset

df_amazon_clean = df_amazon.na.drop("any")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Comprobamos que efectivamente no existen valores nulos

df_amazon_clean.select([count(when(col(c).isNull(), c)).alias(c) for c in df_amazon_clean.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+----------------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|year|product_category|
+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+----------------+
|          0|          0|        0|         0|             0|            0|          0|            0|          0|   0|                0|              0|          0|          0|   0|               0|
+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+----------------+

No se va hacer ninguna revisión de **outliers** pues carece de sentido con la información de este dataset. Sólo cabría en `star_rating` si no oscilan los valores entre 1 y 5 pero se va a comprobar posteriormente con el describe.

Una vez visto este punto, veamos en qué **formato** están cada uno de los campos:

In [5]:
df_amazon_clean.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- product_category: string (nullable = true)

Hay 4 campos que no se adecuan mucho con el formato que son:

- Por un lado está `customer_id` y `product_parent` que estan en formato string (texto) cuando en ellos solo hay números (cosa que por ejemplo no ocurre con `review_id` y `product_id` que se juntan letras y números). Se casteará ambos a **integer** (números enteros).

- En el caso de `vine` y `verified_purchase` no es del todo erróneo que sea string al ser un  "Y" (SÍ) o un "N" (NO) pero ya que solo gira en torno a una variable binaria veo más recomendable transformarlo en **booleanos**, es decir, que sea "true" (Verdad) o "false" (Falso). 

In [6]:
# Casteamos los campos:

df_amazon_casted = (
 df_amazon_clean
    .withColumn("customer_id", f.col("review_id").cast("integer"))
    .withColumn("product_parent", f.col("product_id").cast("integer"))
    .withColumn("vine",f.col("vine").cast("boolean"))
    .withColumn("verified_purchase",f.col("verified_purchase").cast("boolean"))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Vemos cómo ha cambiado

df_amazon_casted.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: boolean (nullable = true)
 |-- verified_purchase: boolean (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- product_category: string (nullable = true)

### Pregunta 2: Obten los valores estadísticos de cada una de las columnas

In [11]:
df_amazon_casted.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+-----------+--------------+--------------------+-------------------+--------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+----------------+
|summary|marketplace|customer_id|     review_id|          product_id|     product_parent|       product_title|       star_rating|     helpful_votes|       total_votes|     review_headline|         review_body|              year|product_category|
+-------+-----------+-----------+--------------+--------------------+-------------------+--------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+----------------+
|  count|  160789452|          0|     160789452|           160789452|           18113815|           160789452|         160789452|         160789452|         160789452|           160789452|           160789452|         160789452|       160789452|
|   mean|       

Realmente los únicos campos con sentido matemático y que nos interesan son los de:

- **`star_rating`**: sus valores se mueven dentro de lo estimado, que es entre 1 y 5. Predominan las valoraciones positivas pues la media de calificación se sitúa en el 4,20.

- **`helpful_votes`**: apenas cada producto tiene una media de dos votos de respuesta útil recibidos. Hay productos que no han recibido ninguna clase de voto y uno que recibió 47.524 votos.

- **`total_votes`**: cada producto tiene aproximadamente una media de 2 y 3 votos, con algún producto que no recibe ninguna y uno que tuvo 48.362.

Como no salen `vine` y `verified_purchase` por el casteo a booleano hacemos el describe de esos dos campos con el dataset original.

In [12]:
df_amazon.describe("vine", "verified_purchase").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+-----------------+
|summary|     vine|verified_purchase|
+-------+---------+-----------------+
|  count|160796570|        160796570|
|   mean|     null|             null|
| stddev|     null|             null|
|    min|        N|                N|
|    max|        Y|                Y|
+-------+---------+-----------------+

### Pregunta 3: Verifica que los campos de star_rating, vine, y verified purchase contienen valores correctos de acuerdo a lo descrito en data layout

Con la información de ambos describe vamos caso por caso:

- **`star_rating_vine`**: sus valores deben comprender entre 1 y 5, que son el número de estrellas con las que se califica cada artículo. Su valor mínimo es 1 y el máximo es 5. Teniendo en cuenta que Amazon no deja valorar con decimales el rango se adecua a lo descrito en el data layout.

- **`vine`**: se identifica si la review ha sido escrita o no en el programa Vine. La respuesta es SI o NO, así que más allá del transformación a booleano comentada anteriormente vemos que ambos valores se mueven entre el "Y" (SÍ) o "N" (NO) por lo también sus valores son correctos.

- **`verified_purchase`**: misma situación que en `vine`, más allá de su casteo a booleano cumple con la descripción del data layout.

### Pregunta 4: Obtén la cantidad total de categorías distintas asociadas a las reviews. ¿Crees que tendría sentido añadir una nueva columna con un menor nivel de granularidad?

In [7]:
# Miramos el número total de categorías distintas

df_amazon_casted.select("product_category").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

43

In [13]:
# Veamos el nombre de las 43 categorías

df_amazon_casted.select("product_category").distinct().show(43, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|product_category        |
+------------------------+
|PC                      |
|Kitchen                 |
|Video                   |
|Digital_Video_Games     |
|Sports                  |
|Outdoors                |
|Grocery                 |
|Home_Improvement        |
|Books                   |
|Mobile_Apps             |
|Furniture               |
|Shoes                   |
|Digital_Music_Purchase  |
|Toys                    |
|Watches                 |
|Jewelry                 |
|Luggage                 |
|Tools                   |
|Baby                    |
|Major_Appliances        |
|Automotive              |
|Gift_Card               |
|Digital_Software        |
|Musical_Instruments     |
|Home                    |
|Health_&_Personal_Care  |
|Digital_Video_Download  |
|Mobile_Electronics      |
|Video_Games             |
|Apparel                 |
|Home_Entertainment      |
|Lawn_and_Garden         |
|Music                   |
|Software                |
|

In [32]:
# A modo de información, sacamos el número de productos que tiene cada una de las categorías

(
    df_amazon_casted
    .select("product_category")
    .groupBy("product_category")
    .agg(
        f.count("product_category").alias("total_products"),
        f.col(count("product_category")/df_amazon_casted.count()
    )
    .orderBy("total_products", ascending = False)
    .show(43, False)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+--------------+
|product_category        |total_products|
+------------------------+--------------+
|Books                   |20725905      |
|Digital_Ebook_Purchase  |19180684      |
|Wireless                |9037709       |
|Video_DVD               |7135754       |
|PC                      |7004142       |
|Mobile_Apps             |6807074       |
|Home                    |6228026       |
|Music                   |6177622       |
|Apparel                 |5905683       |
|Health_&_Personal_Care  |5332715       |
|Digital_Video_Download  |5173648       |
|Beauty                  |5115351       |
|Toys                    |4981218       |
|Kitchen                 |4882582       |
|Sports                  |4859725       |
|Shoes                   |4379038       |
|Automotive              |3515964       |
|Electronics             |3120848       |
|Office_Products         |2646406       |
|Pet_Products            |2643453       |
|Home_Improvement        |2640482 

Respecto a la pregunta planteada, considero que aunque están bastante bien descritas cada una de las categorías algunas de ellas se podrían agrupar en una nueva columna:

- Observamos 5 categorías que empiezan con "Digital" por lo que de alguna manera llamandose `Digital_Products`.

- Lo mismo diría con `Home` en el cual hay 3 categorías que se podrían agrupar todas en la propia `Home`. 

- `Watches` y `Jewelry` las incluiría en una nueva categoría de `Accesories`. 

- `Health_&_Personal_Care` y `Personal_Care_Appliances` pueden juntarse en una misma categoría mismamente.

## Conteo del total de elementos

Una vez eliminados los valores nulos, comprobemos como queda el conteo total (aunque ya se ha visto en el describe):

In [14]:
df_amazon_casted.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

160789452

## Filtrado y copia

Como el dataset es masivo vamos a trabajar únicamente con una partición. Concretamente utilizaremos la partición de `Electronics`.

- Crea un dataframe con el contenido únicamente de la categoría `Electronics` para el mercado americano `US`
- Cuenta el total de elementos de este nuevo dataset
- Guarda esta muestra en un bucket de tu cuenta de AWS
- Vuelve a cargar el dataset desde este bucket

In [28]:
# Creamos el dataframe con las características que queremos

df_electronics = (
    df_amazon_casted
        .where((f.col("product_category") == "Electronics"))
        .where((f.col("marketplace") == "US"))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_electronics.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+----+----------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|year|product_category|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+----+----------------+
|         US|       null|R2RX7KLOQQ5VBG|B00000JBAT|          null|Diamond Rio Digit...|          3|            0|          0|false|            false|Why just 30 minutes?|RIO is really gre...| 1999-06-22|1999|     Electronics|
|         US|       null| RPHMRNCGZF2HN|B001BRPLZU|          null|NG 283220 AC Adap...|         

Llama la atención que habiendo anteriormente eliminado todos los valores nulos ahora aparezcan en `review_id` y `product_id`. **Recordemos que hemos eliminado los valores nulos antes del casteo**, así que vamos a ver si solo estos dos campos que son los que se han transformado a integer se han visto afectados.

In [18]:
df_electronics.select([count(when(col(c).isNull(), c)).alias(c) for c in df_electronics.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+----------------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|year|product_category|
+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+----------------+
|          0|    3105238|        0|         0|       3102514|            0|          0|            0|          0|   0|                0|              0|          0|          0|   0|               0|
+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+----------------+

Efectivamente, parece que **el casteo ha alterado de alguna manera los valores de estos dos campos**. Al igual que la otra vez vamos a comprobar la magnitud de estos valores nulos.

In [23]:
# Porcentaje de customer_id

percentage_null_customer_id = (df_electronics.filter("customer_id is null").count()/df_electronics.count()*100)
percentage_null_customer_id

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100.0

In [24]:
percentage_null_product_parent = (df_electronics.filter("product_parent is null").count()/df_electronics.count()*100)
percentage_null_product_parent

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

99.91227725539879

Absolutamente todos los valores de `customer_id` son nulos y practicamente también en `product_parent`. Si fuese información útil sería recomendable volver a castearlos en el formato anterior, pero según el data layout **son identificadores aleatorios de las reviews sin ninguna relación con el cliente o el producto** por lo que en este caso vamos a proceder a eliminar ambas columnas en su totalidad.

In [32]:
df_electronics = df_electronics.drop("customer_id","product_parent")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

El nuevo dataframe con este cambio quedaría así.

In [33]:
df_electronics.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+----------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+----+----------------+
|marketplace|     review_id|product_id|       product_title|star_rating|helpful_votes|total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|year|product_category|
+-----------+--------------+----------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+----+----------------+
|         US|R2RX7KLOQQ5VBG|B00000JBAT|Diamond Rio Digit...|          3|            0|          0|false|            false|Why just 30 minutes?|RIO is really gre...| 1999-06-22|1999|     Electronics|
|         US| RPHMRNCGZF2HN|B001BRPLZU|NG 283220 AC Adap...|          5|            0|          0|false|             true|          Five Stars|Great quality for...| 2014-11-17|2014|     Electronics|
|    

Como no hemos indicado antes el cuenteo total de elementos lo hacemos ahora.

In [34]:
df_electronics.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3105238

Guardamos en S3 este dataframe.

In [42]:
(
    df_electronics
        .write
        .format("parquet")
        .option("path", f"s3://alex-spark/Proyecto_consolidacion/data_electronics")
        .saveAsTable("data_electronics")
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Una vez guardado, lo abrimos y comprobamos que efectivamente se trata del **mismo dataframe**.

In [7]:
df_electronics_def = (
    spark
        .read
        .format("parquet")
        .option("header", True)
        .load(f"s3://alex-spark/Proyecto_consolidacion/data_electronics")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df_electronics_def.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+----------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+----+----------------+
|marketplace|     review_id|product_id|       product_title|star_rating|helpful_votes|total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|year|product_category|
+-----------+--------------+----------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+----+----------------+
|         US|R2RX7KLOQQ5VBG|B00000JBAT|Diamond Rio Digit...|          3|            0|          0|false|            false|Why just 30 minutes?|RIO is really gre...| 1999-06-22|1999|     Electronics|
|         US| RPHMRNCGZF2HN|B001BRPLZU|NG 283220 AC Adap...|          5|            0|          0|false|             true|          Five Stars|Great quality for...| 2014-11-17|2014|     Electronics|
|    

## Análisis exploratorio inicial

### Pregunta 1: Cuenta el total de artículos conforme al número de estrellas recibidas

In [47]:
(
    df_electronics_def
    .select("product_title", "star_rating")
    .groupBy("star_rating")
    .agg(
        f.count("product_title").alias("total_products") 
    )
    .orderBy("star_rating")
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+
|star_rating|total_products|
+-----------+--------------+
|          1|        359269|
|          2|        179839|
|          3|        239467|
|          4|        538835|
|          5|       1787828|
+-----------+--------------+

**Destacan mayoritariamente respecto al resto los productos calificados con 5 estrellas**, seguidos de los que les puntuaron con 4 estrellas. Los valores entre 2 y 3 estrellas son los que menos productos tienen asociados.

### Pregunta 2: Obtén los 10 productos más votados mostrando su nombre, numero de votos y valoración media

In [48]:
(
  df_electronics_def
    .groupBy("product_title")
    .agg(
        count("total_votes").alias("total_votes"),
         f.mean("star_rating").alias("rating_mean"))
     .withColumn("rating_mean_round", f.round("rating_mean", 2))
     .drop("rating_mean")
    .orderBy("total_votes", ascending = False)
    .show(10, False)
    
)         

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-----------------+
|product_title                                                                                                                                                                                                |total_votes|rating_mean_round|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-----------------+
|Panasonic ErgoFit In-Ear Earbud Headphone                                                                                                                                                                    |24833      |4.41             |
|AmazonBasics High-Speed HDMI Cable - 6.5 Feet (

Exceptuando el `Clip Plus 4 GB MP3 Player (Black)` **todos tienen una muy buena valoración siendo los más votados**, especialmente el `AmazonBasics High-Speed HDMI Cable - 6.5 Feet (2 Meters) Supports Ethernet, 3D, 4K and Audio Return` el cual por cierto como se ve es una variación del `AmazonBasics High Speed HDMI Cable` que también aparece como el sexto más votado y con una puntuación bastante elevada.

### Pregunta 3: Obtén la cantidad de reviews por mes y año y su valoración media

In [49]:
(
   df_electronics_def
       .withColumn("Month", f.month(f.col("review_date")))
       .groupBy("Month")
       .agg(f.count("star_rating").alias("rating_count"),
            f.mean("star_rating").alias("rating_mean"))
       .orderBy("Month")
       .withColumn("rating_mean_round", round("rating_mean", 2))
       .drop("rating_mean")
       .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------+-----------------+
|Month|rating_count|rating_mean_round|
+-----+------------+-----------------+
|    1|      350222|             4.07|
|    2|      282347|             4.05|
|    3|      282857|             4.05|
|    4|      242984|             4.03|
|    5|      238223|             4.03|
|    6|      239647|             4.02|
|    7|      287271|             4.05|
|    8|      297853|             4.05|
|    9|      187226|             4.01|
|   10|      192889|             4.01|
|   11|      204414|             4.01|
|   12|      299305|             4.03|
+-----+------------+-----------------+

Respecto al análisis por meses tenemos estos resultados:

- Donde más reviews se hacen son en **enero**, seguramente coincidiendo con la valoración de los productos que han comprado o les han regalado durante las Navidades. Por el contrario, los meses con menos reviews corresponden a **septiembre, octubre y noviembre**.

- **Apenas hay variación en la valoración media** que oscila entre el 4 y el 4,1 de puntuación. 

In [50]:
(
   df_electronics_def
       .groupBy("year")
       .agg(f.count("star_rating").alias("rating_count"),
            f.mean("star_rating").alias("rating_mean"))
       .orderBy("year")
       .withColumn("rating_mean_round", round("rating_mean", 2))
       .drop("rating_mean")
       .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------+-----------------+
|year|rating_count|rating_mean_round|
+----+------------+-----------------+
|1999|         701|              3.9|
|2000|        4443|             3.97|
|2001|        5483|             3.74|
|2002|        7179|             3.62|
|2003|        9832|             3.61|
|2004|       12601|             3.53|
|2005|       20104|             3.64|
|2006|       29596|              3.7|
|2007|       61414|             3.94|
|2008|       71312|             3.93|
|2009|       91386|             3.92|
|2010|      124245|              3.9|
|2011|      188301|             3.91|
|2012|      270448|             3.98|
|2013|      559101|             4.06|
|2014|      835689|             4.08|
|2015|      813403|             4.11|
+----+------------+-----------------+

Respecto a los años sacamos dos conclusiones:
    
- Como era de esperar con la globalización y el acceso a internet, **el número total de reviews ha ido incrementándose a lo largo de los años progresivamente**. No obstante, es curioso que **en 2014 el número de reviews se duplicó a las de 2013, siendo incluso más altas que en 2015**. Eso último puede ser un indicativo de que estos incrementos de reviews se estén estabilizando (faltaría saber cómo ha variado en años futuros).

- Al principio la valoración fue disminuyendo, pero **desde 2007 se ha ido moviendo entre el 3,9 y el 4,11**. Desde **2011** las valoraciones van a la alza.

### Pregunta 4: Obtén la longitud media, mínima y máxima de la review agrupado por la cantidad de votos de respuesta útil recibidos (helpful_votes), ordenala según el número de votos recibidos

__Pistas:__
    - Separar por palabras la descripcion (split)
    - Contar el tamaño del vector resultante (size)

In [12]:
(
    df_electronics_def
    .select("review_body", "total_votes","helpful_votes")
    .withColumn("review_body_split", f.split(df_electronics_def.review_body," "))
    .withColumn("review_size",f.size(col("review_body_split")))
    .groupBy("helpful_votes")
    .agg(
    f.mean("review_size").alias("review_size_mean"),
    f.min("review_size").alias("review_size_min"),
    f.max("review_size").alias("review_size_max"),
    f.count("total_votes").alias("total_votes")
    )
    .orderBy("total_votes", ascending = False)
    
    # Eliminamos los decimales para aproximar el número exacto de palabras medias 
    
    .withColumn("review_size_mean_round", round("review_size_mean", 0))
    .drop("review_size_mean")
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+---------------+---------------+-----------+----------------------+
|helpful_votes|review_size_min|review_size_max|total_votes|review_size_mean_round|
+-------------+---------------+---------------+-----------+----------------------+
|            0|              1|           5915|    2098640|                  46.0|
|            1|              1|           5986|     472080|                  81.0|
|            2|              1|           7405|     173742|                 109.0|
|            3|              1|           3986|      91988|                 130.0|
|            4|              1|           5413|      56122|                 144.0|
|            5|              1|           3529|      38082|                 156.0|
|            6|              1|           5694|      27120|                 165.0|
|            7|              1|           5122|      20294|                 173.0|
|            8|              1|           5491|      15539|                 182.0|
|   

Lo único a destacar de este punto es que a medida que las reviews cuentan con mayor número de votos de respuesta útil recibidos, **el número total de votoso recibidos disminuye y la media de palabras por review aumenta**.

### Pregunta 5: Obtén la cantidad de reviews para cada una de las posibles valoraciones (star_rating), ¿crees que si quisiéramos implementar un modelo para predecir el precio de nuevos productos tendríamos algún problema?

In [52]:
(
    df_electronics_def
    .select("review_headline", "star_rating")
    .groupBy("star_rating")
    .agg(
        f.count("review_headline").alias("total_reviews") 
    )
    .orderBy("star_rating")
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+
|star_rating|total_reviews|
+-----------+-------------+
|          1|       359269|
|          2|       179839|
|          3|       239467|
|          4|       538835|
|          5|      1787828|
+-----------+-------------+

Como se puede apreciar, el resultado es idéntico al expuesto en la pregunta 1. Esto es así porque **cada review o reviews se asocian a sus respectivos productos**, por lo que el número total de reviews para cada calificación siempre va a coincidir con el número total de productos.

## Transformaciones básicas de algunos campos

### Pregunta: Quita de los campos que contienen descripciones del contenido de la review aquellos caracteres que no aportan información, como !, [, o las etiquetas HTML

Veamos como actualmente se ven `review_body` y `review_headline`. 

In [17]:
(
    df_electronics_def
    .select("review_body", "review_headline")
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|         review_body|     review_headline|
+--------------------+--------------------+
|RIO is really gre...|Why just 30 minutes?|
|Great quality for...|          Five Stars|
|One of several fi...|The digital audio...|
|I like it, got so...|          Five Stars|
|I returned mine f...|Design flaws ruin...|
|Never got around ...|Never got around ...|
|I really like the...| Might want to wait.|
|Saved my.marriage...|Saved my. marriag...|
|Very Good Product...|LOW PRICE GOOD VALUE|
|thanks to this i ...|          Five Stars|
|I've had this rec...|STR-DE525 an over...|
|I only used half ...|but it worked per...|
|This is a absolut...|Incredible Sound ...|
|These batteries t...|Quality and long ...|
|Someday everythin...|The wave of the f...|
|Even though they ...|Even though they ...|
|This is the sixth...|         Poor choice|
|The ideal thing t...|Easy to unpluged ...|
|I just got this n...|     BUY THIS!!!!!!!|
|           excellent|          

Aunque no podemos ver las reviews en su totalidad (al intentar ponerlas enteras no se podrían apreciar sin hacer la pantalla excesivamente pequeña lo que lo haría imposible de leer) al menos vemos que **una review tiene signos de exclamación**, signo el cual queremos eliminar.

In [18]:
# Definimos cada regex_string con el signo que vamos a querer eliminar

regex_string_exclamation = "\!"
regex_string_bracket = "\]"
regex_string_HTML = "<[^>]+>"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Creamos un nuevo dataframe con todos estos signos eliminados

df_electronics_def_cleaned = (df_electronics_def 

      .withColumn("review_body", f.regexp_replace(f.col("review_body"), regex_string_exclamation, "").alias("review_body_clean"))
      .withColumn("review_body_clean", f.regexp_replace(f.col("review_body"), "\"", "").alias("review_body_clean"))
    
      .withColumn("review_body", f.regexp_replace(f.col("review_body"), regex_string_bracket, "").alias("review_body_clean"))
      .withColumn("review_body_clean", f.regexp_replace(f.col("review_body"), "\"", "").alias("review_body_clean"))
    
    
      .withColumn("review_body", f.regexp_replace(f.col("review_body"), regex_string_HTML, "").alias("review_body_clean"))
      .withColumn("review_body_clean", f.regexp_replace(f.col("review_body"), "\"", "").alias("review_body_clean"))

    
      .withColumn("review_headline", f.regexp_replace(f.col("review_headline"), regex_string_exclamation, "").alias("review_headline_clean"))
      .withColumn("review_headline_clean", f.regexp_replace(f.col("review_headline"), "\"", "").alias("review_headline_clean"))
    
      .withColumn("review_headline", f.regexp_replace(f.col("review_headline"), regex_string_bracket, "").alias("review_headline_clean"))
      .withColumn("review_headline_clean", f.regexp_replace(f.col("review_headline"), "\"", "").alias("review_headline_clean"))
    
      .withColumn("review_headline", f.regexp_replace(f.col("review_headline"), regex_string_HTML, "").alias("review_headline_clean"))
      .withColumn("review_headline_clean", f.regexp_replace(f.col("review_headline"), "\"", "").alias("review_headline_clean"))
    
      .drop("review_body","review_headline")
      .select("review_body_clean","review_headline_clean")
                             )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Veamos sí se han eliminado o no aquellos signos de exclamación.

In [21]:
(
    df_electronics_def_cleaned
    .select("review_body_clean", "review_headline_clean")
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+
|   review_body_clean|review_headline_clean|
+--------------------+---------------------+
|RIO is really gre...| Why just 30 minutes?|
|Great quality for...|           Five Stars|
|One of several fi...| The digital audio...|
|I like it, got so...|           Five Stars|
|I returned mine f...| Design flaws ruin...|
|Never got around ...| Never got around ...|
|I really like the...|  Might want to wait.|
|Saved my.marriage...| Saved my. marriag...|
|Very Good Product...| LOW PRICE GOOD VALUE|
|thanks to this i ...|           Five Stars|
|I've had this rec...| STR-DE525 an over...|
|I only used half ...| but it worked per...|
|This is a absolut...| Incredible Sound ...|
|These batteries t...| Quality and long ...|
|Someday everythin...| The wave of the f...|
|Even though they ...| Even though they ...|
|This is the sixth...|          Poor choice|
|The ideal thing t...| Easy to unpluged ...|
|I just got this n...|             BUY THIS|
|         

**Se han eliminado completamente** por lo que confirmamos que este nuevo dataframe está limpio de aquellos signos que no nos interesaban.

## Análisis más profundo

### Pregunta 1: Obtén la media de calificación por categoría para aquellas compras verificadas en los mercados francés, español y alemán de los últimos 10 años

En el data layout **no se indican explícitamente qué mercados** están en el dataset así que vamos a descubrirlo por nuestra cuenta.

In [27]:
df_amazon.select("marketplace").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|marketplace|
+-----------+
|         DE|
|         US|
|         UK|
|         FR|
|         JP|
+-----------+

Sí que aparecen los mercados francés (`FR`) y alemán (`DE`) pero **no el español**, asi que de ese mercado no podemos calcular nada en este Proyecto.

Empecemos con el **mercado francés**.

In [25]:
(
    df_amazon_casted
    .select("product_category", "year", "star_rating", "marketplace", "verified_purchase")
    .where((f.col("marketplace") == "FR"))
    .where((f.col("verified_purchase") == "true"))
    .where((f.col("year") > 2005))
    .groupBy("product_category").pivot("year")
    .agg(f.mean("star_rating"))
    .orderBy("product_category", ascending = False)
    .show(40, False)
    
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|product_category        |2006             |2007             |2008             |2009              |2010              |2011              |2012              |2013              |2014              |2015              |
+------------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|Wireless                |null             |null             |null             |null              |5.0               |4.458333333333333 |3.774193548387097 |3.8787061994609164|3.898107714701601 |3.75177304964539  |
|Watches                 |null             |null             |null             |4.0               |4.5               |3.8125            |4.21978

Como podemos observar, sabiendo que el tenemos un total de 43 cateogorías de productos, **en este mercado aparece un número inferior**. A destacar:

- **Tan solo** hay registros en los primeros años de DVD (`Video_DVD`), productos relacionados con la música (`Music`) y libros (`books`). Hasta **2012** no hay registro de todos los distintos productos recogidos.

- Se observa una tendencia de **disminución de la valoración media a medida que van pasando los años**.

- No obstante, encontramos algunas **excepciones** como con `Personal_Care_Appliances`, `Office_Products`, `Luggage`, `Kitches` y `Automative`. 

- Los productos relacionados con la música (`Music`) son los que tienen la **valoración media más estable del resto de categorías**.

In [31]:
(
    df_amazon_casted
    .select("product_category", "year", "star_rating", "marketplace", "verified_purchase")
    .where((f.col("marketplace") == "DE"))
    .where((f.col("verified_purchase") == "true"))
    .where((f.col("year") > 2005))
    .groupBy("product_category").pivot("year")
    .agg(f.mean("star_rating"))
    .orderBy("product_category", ascending = False)
    .show(40, False)
    
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|product_category        |2006             |2007              |2008             |2009              |2010              |2011              |2012              |2013              |2014              |2015              |
+------------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|Wireless                |5.0              |5.0               |4.5              |4.375             |3.63265306122449  |3.49              |3.6204188481675392|3.842487046632124 |3.8253182461103252|3.9690402476780187|
|Watches                 |null             |null              |2.0              |4.125             |4.4               |4.243243243243243 |4.

Al igual que con el mercado francés, el aleman **tampoco presenta todas las categorías de productos existentes** (hay algunas ligeras diferencias como que aquí aparece la categoría `Software`, `Digital_Video_Download` o `Furniture` cuando en el otro mercado no). Resaltamos lo siguiente, lo cual también coincide mayoritariamente con lo antes expuesto:

- **Solamente** hay registros en los primeros años de DVD (`Video_DVD`), productos relacionados con la música (`Music`) y libros (`books`).

- Hay una ligera tendencia a tener peor valoración media con respecto pasan los años, **aunque ocurre con menos productos que en el mercado francés**. Sin embargo, en 2015 la mayoría de valoraciones **son más altas en Alemania** que en Francia frente al mismo producto.

- Una valoración perfecta en **`Grocery`** teniendo en cuenta que en 2014 ni hay registros hace sospechar que o apenas hay productos con estas categorías o son productos con escasas valoraciones y todas positivas.

### Pregunta 2: Define una columna llamada "reviewValidity" que contenga el valor "valid" en caso de que la review la consideres válida, en caso contrario, asociale el valor "invalid"

In [8]:
# Creamos un nuevo dataset con la nueva columna

df_amazon_complete = (
    df_amazon_casted.withColumn("reviewValidity", f.when(f.col("verified_purchase") == "true", f.lit("valid")).otherwise(f.lit("invalid"))
                        )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Mostremos el dataset con la nueva columna.

In [51]:
df_amazon_complete.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+---------+----------+--------------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+----+----------------+--------------+
|marketplace|customer_id|review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|year|product_category|reviewValidity|
+-----------+-----------+---------+----------+--------------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+----+----------------+--------------+
|         US|   15444933|     null|1848192576|     835940987|Standing Qigong f...|          5|            9|         10|false|             true|Informative AND i...|After attending a...| 2015-05-02|2015|           Books|         valid|
|         US|   20595117|     null|      null|     57404

### Pregunta 3: ¿Qué marketplaces tienen un mayor volumen de reviews no válidas por año? Por otro lado, ¿cuáles contienen más reviews inválidas?

In [55]:
# Primero con las reviews no válidas

(
    df_amazon_complete
    .select("marketplace", "reviewValidity")
    .where((f.col("reviewValidity") == "invalid")) 
    .groupBy("marketplace")
    .agg(
        f.count("reviewValidity").alias("invalid_review")
    )
    .orderBy("invalid_review", ascending = False)
    .show()
    
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+
|marketplace|invalid_review|
+-----------+--------------+
|         US|      35120356|
|         UK|        406441|
|         DE|        295220|
|         JP|        123495|
|         FR|        100895|
+-----------+--------------+

In [8]:
# Seguimos con las reviews válidas

(
    df_amazon_complete
    .select("marketplace", "reviewValidity")
    .where((f.col("reviewValidity") == "valid")) 
    .groupBy("marketplace")
    .agg(
        f.count("reviewValidity").alias("valid_review")
    )
    .orderBy("valid_review", ascending = False)
    .show()
    
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------+
|marketplace|valid_review|
+-----------+------------+
|         US|   122773088|
|         UK|     1301053|
|         DE|      383901|
|         FR|      153185|
|         JP|      138936|
+-----------+------------+

Tanto para las reviews válidas como las inválidas destaca **Estados Unidos (`US`)**. Tanto para reviews válidas como inválidas le siguen Reino Unido (`UK`) y Alemania (`DE`), cambiando las posiciones entre el mercado alemán (`DE`) y el japonés (`JP`).

### Pregunta 4: Normalmente, cuando repasamos rápidamente una review en Amazon, miramos al título directamente. En este sentido, para cada una de las categorías con valoraciones de 4 o 5 estrellas, cuáles son las 10 palabras más empleadas en el título?

In [40]:
(
   df_amazon_complete
     .select("review_headline", "star_rating")
     .where((f.col("star_rating") > 4)) 
     .withColumn("splitted", f.split(f.col("review_headline"), " "))
     .withColumn("exploded", f.explode(f.col("splitted")))
     .groupBy("exploded")
     .agg(
         f.count("exploded").alias("frequent_words")
     )
    .orderBy("frequent_words", ascending = False)
    .show(10)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+
|exploded|frequent_words|
+--------+--------------+
|   Stars|      18398986|
|    Five|      18392870|
|   Great|       8770470|
|     the|       7823036|
|     and|       7193485|
|     for|       6263120|
|       a|       5444894|
|       I|       5144953|
|      to|       4597017|
|      of|       4290875|
+--------+--------------+
only showing top 10 rows

La mayoría de las palabras son pronombres (`I`), conjunciones (`and`) o determinaciones (`the`); por lo que **vamos a extender la lista a las 50 palabras más empleadas**.

In [41]:
(
   df_amazon_complete
     .select("review_headline", "star_rating")
     .where((f.col("star_rating") > 4)) 
     .withColumn("splitted", f.split(f.col("review_headline"), " "))
     .withColumn("exploded", f.explode(f.col("splitted")))
     .groupBy("exploded")
     .agg(
         f.count("exploded").alias("frequent_words")
     )
    .orderBy("frequent_words", ascending = False)
    .show(50) # Lo extendemos ahora a 50
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------+
| exploded|frequent_words|
+---------+--------------+
|    Stars|      18398986|
|     Five|      18392870|
|    Great|       8770470|
|      the|       7823036|
|      and|       7193485|
|      for|       6263120|
|        a|       5444894|
|        I|       5144953|
|       to|       4597017|
|       of|       4290875|
|    great|       3906629|
|       it|       3546778|
|     this|       3294303|
|       is|       3269656|
|        A|       3249048|
|      ...|       3019693|
|     Love|       2347355|
|     love|       2335519|
|      The|       2311157|
|       my|       2203880|
|     book|       2099135|
|     good|       2081524|
|       in|       2073677|
|     Good|       1977885|
|     with|       1722218|
|     Very|       1621271|
|Excellent|       1570094|
|     This|       1511593|
|       on|       1389790|
|     Best|       1373488|
|  product|       1278473|
|      you|       1248807|
|     very|       1225065|
|     best|       1203862|
|

Aunque aparecen más pronombres y conjunciones ahora encontramos **más adjetivos que antes**: `Great` aparece 2 veces, así como `Love`. Aparecen nuevos adjetivos como `Best`, `Excellent` o `Awesome`. Destaca dentro de productos, además de la propia palabra `product` aparece una de sus categorías como son los libros (`books`).

### Pregunta 5: Otro aspecto que sí que comparten las reviews son el título del producto. En este sentido, cual es el top 10 de productos mejor valorados del dataset, siempre y cuando consideremos las reviews válidas. De esos 10 productos, ¿existen diferencias en cuanto al rating que tienen en cada marketplace? ¿Qué información podría extraer la marca si se añade la componente temporal a ese análisis?

In [24]:
(
    df_amazon_complete
        .where((f.col("reviewValidity") == "valid"))
        .groupBy("marketplace", "product_title")
        .agg(
            f.mean(f.col("star_rating")).alias("rating_mean"),
            f.sum(f.col("total_votes")).alias("sum_total_votes")
        )
        .withColumn("rating_mean_round", round("rating_mean", 2))
        .drop("rating_mean")
        .orderBy("sum_total_votes","rating_mean_round", ascending = False)
        .limit(10)
        .show(10, False)
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------------------------------------------------------------------------------+---------------+-----------------+
|marketplace|product_title                                                                           |sum_total_votes|rating_mean_round|
+-----------+----------------------------------------------------------------------------------------+---------------+-----------------+
|US         |Minecraft                                                                               |877403         |4.56             |
|US         |Subway Surfers                                                                          |338626         |4.66             |
|US         |Minion Rush: Despicable Me Official Game                                                |200860         |4.46             |
|US         |Pixel Gun 3D (Pocket Edition) - multiplayer shooter with skin creator                   |182492         |4.52             |
|US         |Kindle Fire (Previous Genera

Lo primero que tenemos que decir que debido al enorme impacto que tiene **Estados Unidos (`US`)** respecto al resto hace imposible que con solo 10 productos podamos encontrar marketplaces distintos a este. Sólamente se exeptúa el videojuego de **`Minecraft`** en Reino Unido (`UK`) el cual es el líder supremo en votos pero esta vez en el mercado americano. 

La influencia creo que puede dar añadir una componente temporal creo que es clara: **si el producto es una moda**. Realmente si nos fijamos la mayoría de productos de este top 10 son videojuegos (ya sean de plataformas o de dispositivos móviles y tablets). En esta época de la digitalización (2015) encaja perfectamente que los videojuegos destaquen sobre otra clase de productos. Por ello, vamos a condicionar este top 10 solo hasta el **año 2005** donde los resultados no estaban tanto en tendencia a ver qué resultados encontramos.

In [28]:
(
    df_amazon_complete
        .where((f.col("reviewValidity") == "valid"))
        .where((f.col("year") < 2005)) # Añadimos la nueva condición
        .groupBy("marketplace", "product_title")
        .agg(
            f.mean(f.col("star_rating")).alias("rating_mean"),
            f.sum(f.col("total_votes")).alias("sum_total_votes")
        )
        .withColumn("rating_mean_round", round("rating_mean", 2))
        .drop("rating_mean")
        .orderBy("sum_total_votes","rating_mean_round", ascending = False)
        .limit(10)
        .show(10, False)
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------------------------------------------------------------------------+---------------+-----------------+
|marketplace|product_title                                                                    |sum_total_votes|rating_mean_round|
+-----------+---------------------------------------------------------------------------------+---------------+-----------------+
|US         |A Game of Thrones (A Song of Ice and Fire, Book 1)                               |14670          |4.16             |
|US         |Firefly: The Complete Series                                                     |14236          |4.92             |
|US         |The Da Vinci Code                                                                |10487          |3.82             |
|US         |Unfit For Command: Swift Boat Veterans Speak Out Against John Kerry              |6105           |4.78             |
|US         |The Power of Now: A Guide to Spiritual Enlightenment                         

Como era de esperar, Estados Unidos seguiría destacando pero no así los productos. Más allá de los libros, el producto estrella aquí son las **series y películas**. Así que a la marca le interesa mucho el factor temporal para saber qué sectores están en alza y poder sacar beneficio.

In [ ]:
# En términos globales vemos que media de star_rating tiene cada país

(
    df_amazon_complete
        .where((f.col("reviewValidity") == "valid"))
        .groupBy("marketplace") # Eliminamos product_title al no poder hacerlo de un top 10
        .agg(
            f.mean(f.col("star_rating")).alias("rating_mean"),
            f.sum(f.col("total_votes")).alias("sum_total_votes")
        )
        .withColumn("rating_mean_round", round("rating_mean", 2))
        .drop("rating_mean")
        .orderBy("sum_total_votes","rating_mean_round", ascending = False)
        .show()
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------+-----------------+
|marketplace|sum_total_votes|rating_mean_round|
+-----------+---------------+-----------------+
|US         |213450806      |4.23             |
|UK         |2600482        |4.45             |
|DE         |1528515        |4.37             |
|JP         |1300423        |4.28             |
|FR         |390688         |4.31             |
+-----------+---------------+-----------------+

Aunque incluye todos los productos y no el top 10 (por las dificultades ya planteadas), vemos que en términos generales la diferencia de media de `star_rating` entre sus marketplace **es bastante corta**. Partiendo de que no hay apenas distinciones entre ellos con esa cantidad total tan distinta de votos totales, es probable que esa escasa diferencia de media también se den en un top 10 aunque no se puede asegurar.

## Movimiento de datos

### Almacena el contenido del dataframe de electronics en S3, particionado por año

In [11]:
(
    df_electronics_def
        .write
        .mode("overwrite")
        .partitionBy("year")
        .format("parquet")
        .option("path", f"s3://alex-spark/Proyecto consolidacion/dataset_partitioned")
        .saveAsTable("dataset_partitioned")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Almacena el contenido de un dataframe, agrupado por mercado, año y categoría, de la media de star_rating y el total de reviews pariticionado por año y mes

En este caso, a diferencia del anterior partición, lo haremos en **dos partes**. Primero creamos un dataframe con las diferentes exigencias planteadas.

In [13]:
df_partitioned = (
    df_electronics_def
        .withColumn("year", f.year(f.col("review_date")))
        .withColumn("month", f.month(f.col("review_date")))
        .groupBy("marketplace", "year", "month", "product_category")
        .agg(
            f.avg(f.col("star_rating")).alias("avg_star_rating"),
            f.count(f.col("review_id")).alias("review_count")
        )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Una vez hecho, lo almacenamos.

In [16]:
(
    df_partitioned 
      .write
      .partitionBy("year")
      .mode("overwrite")
      .parquet("s3://alex-spark/Proyecto consolidacion/dataset_partitioned_complete")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Data Scientist expertise

### Pregunta 1: ¿Crees ques sería interesante cruzar este dataset con otros para enriquecerlo? ¿Con cuáles? ¿Por qué?

Generalmente un mismo producto se suele vender en varios lugares, no solo en Amazon. Es por ello que respecto a las reviews hay algunas plataformas como **Fnac o MediaMark que tienen un método de valoración idéntico** al analizado de 5 estrellas. Eso nos podría hacer multitud de **comparaciones** entre distintas tiendas respecto a un mismo producto. Cierto es que no se podría hacer con España pero Fnac tiene tiendas en Francia y MediaMark en el resto de países europeos incluido la misma Francia por lo que sería de utilidad.

### Pregunta 2: Plantea 3 casos de uso diferentes de productos basados en datos a partir del dataset inicial de reviews de Amazon, y cómo tu producto añadiría valor al sector concreto en el que lo enmarcas

Se me ocurren 3 posibles casos:

- **Un dataset de libros**: como hemos visto, es los libros (`books`) son el producto estrella de este dataset. No solo es la categoría con más productos, sino que en los mercados analizados es un producto del que siempre se han hecho reviews desde edades tempranas de Amazon. Seguro que se podríamos agruparlos por sus diferentes géneros con el fin de saber cuáles son los más demandados tanto en términos generales como por población dentro de cada marketplace. También sería interesante si afectan otros factores socio-demográficos como la edad o el sexo a la hora de la elección.

- **Un dataset de productos con reviews negativas estadounidenses**: Estados Unidos (`US`) es unos de los principales mercados del mundo, afirmación que ha quedado reflejada en en el presente dataset de Amazon. Cuanto más contentos estén los clientes de dicho país, más beneficios se van a conseguir que si se hiciese en muchos otros países. La intención de este dataset sería revisar el o los motivos de descontento del cliente para intentar enfocar de otra manera dicho producto o centrarse más en otros productos relacionados pero con mejores reviews.

- **Un dataset con las categorías con mejor media de `star_rating` de cada categoría**: por norma general lo más valorado es lo más vendido, así que las tiendas especializadas en un sector o categoría en concreto van a estar interesados en esos productos de su ámbito que mejor valoración media tenga. Esto sirve tanto para empresas emergentes que parten de lo más apoyado para intentar empezar con menos costes fijos iniciales, empresas ya establecidas que quieran aumentar sus beneficios o incluso empresas que estuviesen pensando en cambiarse de sector y vean una nueva oportunidad de mercado.

### Pregunta 3: ¿Qué otra columna o columnas crees que serían útiles en los datos, de tipo timestamp por ejemplo, para realizar análisis más complejos o profundos?

Añadiría las siguientes columnas:
    
- **`purchase_date`**: sería una columna del tipo de timestamp en el cual se registra la hora y minuto en el que se ha comprado el producto. Sería interesante añadirla porque en el presente dataset están las fechas de las reviews pero no sabemos cuándo lo compro. Por un lado ayudaría a saber en qué horarios los clientes consumen más a diario así como ver el periodo que hay entre que compran el producto y realizan una review.

- **`frequent_words`**: al igual que hemos hecho antes, sería una columna tipo string filtrado con las palabras más empleadas en las reviews. Ahora bien, sería eliminando todas las conjunciones, pronombres, determinantes y verbos que no aportan nada de información, dejando las palabras relacionadas con adjetivos, adverbios o nombres de productos.

- **`state_place`**: sería una columna tipo string relacionada con `marketplace` siendo más especifica en la compra. En España indicaria las Comunidades Autónomas, en Estados Unidos los diferentes Estados o en Alemania  sus Estados Federados, entre otros países.

- **`devolution_purchase`**: consistiría en una columna tipo booleana donde se indicaría si el cliente devolvió o no el producto. 

- Como ya se ha comentado previamente, una columna en la cual estén agrupadas algunas de las categorías de productos con características similares.

**Espero que les haya sido de ayuda este proyecto y deseo que sigan contando con mis servicios en futuros informes de análisis de datos**.